# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 21 2023 11:10AM,257829,10,9187489,"Virtue Rx, LLC",Released
1,Feb 21 2023 10:50AM,257828,19,9187479,"Emersa Waterbox, LLC",Released
2,Feb 21 2023 10:50AM,257828,19,9187480,"Emersa Waterbox, LLC",Released
3,Feb 21 2023 10:48AM,257826,16,SHL-9187475,"SHL Pharma, LLC",Released
4,Feb 21 2023 10:43AM,257825,21,641953,"NBTY Global, Inc.",Released
5,Feb 21 2023 10:36AM,257824,10,9186418,Eye Pharma Inc,Released
6,Feb 21 2023 10:32AM,257823,21,641831,"NBTY Global, Inc.",Released
7,Feb 21 2023 10:31AM,257822,10,37703,"Digital Brands, LLC",Released
8,Feb 21 2023 10:31AM,257822,10,37704,"Digital Brands, LLC",Released
9,Feb 21 2023 10:31AM,257822,10,37705,"Digital Brands, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,257824,Released,1
35,257825,Released,1
36,257826,Released,1
37,257828,Released,2
38,257829,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257824,NaN,NaN,1.0
257825,NaN,NaN,1.0
257826,NaN,NaN,1.0
257828,NaN,NaN,2.0
257829,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257735,0.0,3.0,19.0
257739,0.0,0.0,19.0
257741,0.0,0.0,1.0
257742,0.0,1.0,0.0
257746,0.0,0.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257735,0,3,19
257739,0,0,19
257741,0,0,1
257742,0,1,0
257746,0,0,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257735,0,3,19
1,257739,0,0,19
2,257741,0,0,1
3,257742,0,1,0
4,257746,0,0,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257735,,3,19
1,257739,,,19
2,257741,,,1
3,257742,,1,
4,257746,,,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 21 2023 11:10AM,257829,10,"Virtue Rx, LLC"
1,Feb 21 2023 10:50AM,257828,19,"Emersa Waterbox, LLC"
3,Feb 21 2023 10:48AM,257826,16,"SHL Pharma, LLC"
4,Feb 21 2023 10:43AM,257825,21,"NBTY Global, Inc."
5,Feb 21 2023 10:36AM,257824,10,Eye Pharma Inc
6,Feb 21 2023 10:32AM,257823,21,"NBTY Global, Inc."
7,Feb 21 2023 10:31AM,257822,10,"Digital Brands, LLC"
37,Feb 21 2023 10:12AM,257821,19,Eli Lilly and Company
38,Feb 21 2023 10:10AM,257820,10,"CLINUVEL, Inc."
39,Feb 21 2023 10:09AM,257819,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 21 2023 11:10AM,257829,10,"Virtue Rx, LLC",,,1
1,Feb 21 2023 10:50AM,257828,19,"Emersa Waterbox, LLC",,,2
2,Feb 21 2023 10:48AM,257826,16,"SHL Pharma, LLC",,,1
3,Feb 21 2023 10:43AM,257825,21,"NBTY Global, Inc.",,,1
4,Feb 21 2023 10:36AM,257824,10,Eye Pharma Inc,,,1
5,Feb 21 2023 10:32AM,257823,21,"NBTY Global, Inc.",,,1
6,Feb 21 2023 10:31AM,257822,10,"Digital Brands, LLC",,,30
7,Feb 21 2023 10:12AM,257821,19,Eli Lilly and Company,,,1
8,Feb 21 2023 10:10AM,257820,10,"CLINUVEL, Inc.",,,1
9,Feb 21 2023 10:09AM,257819,10,"CLINUVEL, Inc.",,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 11:10AM,257829,10,"Virtue Rx, LLC",1,,
1,Feb 21 2023 10:50AM,257828,19,"Emersa Waterbox, LLC",2,,
2,Feb 21 2023 10:48AM,257826,16,"SHL Pharma, LLC",1,,
3,Feb 21 2023 10:43AM,257825,21,"NBTY Global, Inc.",1,,
4,Feb 21 2023 10:36AM,257824,10,Eye Pharma Inc,1,,
5,Feb 21 2023 10:32AM,257823,21,"NBTY Global, Inc.",1,,
6,Feb 21 2023 10:31AM,257822,10,"Digital Brands, LLC",30,,
7,Feb 21 2023 10:12AM,257821,19,Eli Lilly and Company,1,,
8,Feb 21 2023 10:10AM,257820,10,"CLINUVEL, Inc.",1,,
9,Feb 21 2023 10:09AM,257819,10,"CLINUVEL, Inc.",2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 11:10AM,257829,10,"Virtue Rx, LLC",1,,
1,Feb 21 2023 10:50AM,257828,19,"Emersa Waterbox, LLC",2,,
2,Feb 21 2023 10:48AM,257826,16,"SHL Pharma, LLC",1,,
3,Feb 21 2023 10:43AM,257825,21,"NBTY Global, Inc.",1,,
4,Feb 21 2023 10:36AM,257824,10,Eye Pharma Inc,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 11:10AM,257829,10,"Virtue Rx, LLC",1.0,NaN,NaN
1,Feb 21 2023 10:50AM,257828,19,"Emersa Waterbox, LLC",2.0,NaN,NaN
2,Feb 21 2023 10:48AM,257826,16,"SHL Pharma, LLC",1.0,NaN,NaN
3,Feb 21 2023 10:43AM,257825,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Feb 21 2023 10:36AM,257824,10,Eye Pharma Inc,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 11:10AM,257829,10,"Virtue Rx, LLC",1.0,0.0,0.0
1,Feb 21 2023 10:50AM,257828,19,"Emersa Waterbox, LLC",2.0,0.0,0.0
2,Feb 21 2023 10:48AM,257826,16,"SHL Pharma, LLC",1.0,0.0,0.0
3,Feb 21 2023 10:43AM,257825,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Feb 21 2023 10:36AM,257824,10,Eye Pharma Inc,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5671697,350.0,0.0,0.0
12,257810,1.0,0.0,0.0
15,1288785,80.0,10.0,0.0
16,515568,1.0,1.0,0.0
19,773443,11.0,12.0,2.0
21,515648,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5671697,350.0,0.0,0.0
1,12,257810,1.0,0.0,0.0
2,15,1288785,80.0,10.0,0.0
3,16,515568,1.0,1.0,0.0
4,19,773443,11.0,12.0,2.0
5,21,515648,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,350.0,0.0,0.0
1,12,1.0,0.0,0.0
2,15,80.0,10.0,0.0
3,16,1.0,1.0,0.0
4,19,11.0,12.0,2.0
5,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,350.0
1,12,Released,1.0
2,15,Released,80.0
3,16,Released,1.0
4,19,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21
Status,,,,,,
Completed,0.0,0.0,0.0,0.0,2.0,0.0
Executing,0.0,0.0,10.0,1.0,12.0,0.0
Released,350.0,1.0,80.0,1.0,11.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,0.0,0.0,10.0,1.0,12.0,0.0
2,Released,350.0,1.0,80.0,1.0,11.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21
0,Completed,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,0.0,0.0,10.0,1.0,12.0,0.0
2,Released,350.0,1.0,80.0,1.0,11.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()